<h1>Chapter 3 | <b>Extra</b> Data Exercise |  EDA With Football Data</h1>

<p>Inspired by the analysis of home team and away team performances of the book, I decided to reproduce a few of these steps using a different dataset. To do that, I will tap into a great API - Football API - and get some statistics on the main Brazilian Football League (Serie A). This will require going back a few steps and review the first two chapters as well. Succinticly, we will do the following procedures here:</p>
<ul>
    <li>Get the data</li>
    <li>Clean the data</li>
    <li>Explore the data</li>
</ul>
<p>This is an opportunity to learn more about how to get data from APIs. Even though this API is packed with data, let's keep it simple and try to do a simple job. With no further ado, let's do it.</p>
<hr>
<h2>Get the data</h2>
<p>Let's import the required libraries and set up our working directories, our yaml variables, and our urls.</p>


In [2]:
import requests
import yaml
import pandas as pd
import os


In [6]:
# Define working directory
dirname = os.getcwd()
data_out = f"{dirname}/data/output/" 

# Define variables set in yaml
with open(f"{dirname}/config/config.yaml") as f:
    config = yaml.safe_load(f)

API_KEY = config["api_key"]
url = "https://v3.football.api-sports.io/"

<p>We can now make a simple function that will make an API call to the url and, using the defined parameters, will return the data in a .json format. It's important - and that goes for any API - to take a look at the <b>documentation</b> to get acquainted with the required enpoints. We need the following information from the API:</p>
<ul>
    <li>All the matches between all teams for the 2022 Serie A season.</li>
    <li>The number of goals scored by team.</li>
    <li>The name of the home and the away team for each match.</p>
</ul>
<p>After reading the docs, I decided that the enpoint <code>/fixtures</code> will help me on this job. Also, to get the required season year and the id of this particular football league, I took a look at the id numbers provided by the API's <a href="https://dashboard.api-football.com/soccer/ids">dashboard</a>.</p>